In [2]:
import h2o
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.automl import H2OAutoML
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 21.0.1+12-29, mixed mode, sharing)
  Starting server from C:\Users\byan\eeg_project\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\byan\AppData\Local\Temp\tmplja7wlx7
  JVM stdout: C:\Users\byan\AppData\Local\Temp\tmplja7wlx7\h2o_byan_started_from_python.out
  JVM stderr: C:\Users\byan\AppData\Local\Temp\tmplja7wlx7\h2o_byan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,2 months and 22 days
H2O_cluster_name:,H2O_from_python_byan_x4ba65
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.931 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [3]:
features_train = h2o.upload_file("data/features_train.csv",header=1)
features_test = h2o.upload_file("data/features_test.csv",header=1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:




# Identify predictors and response
x = features_train.columns
y = "108"
x.remove(y)


# For binary classification, response should be a factor
features_train[y] = features_train[y].asfactor()
features_test[y] = features_test[y].asfactor()
# Load data



# Apply PCA
pca = H2OPrincipalComponentAnalysisEstimator(k = 10) # Set k to the number of components you want
pca.train(x=x, training_frame=features_train)

# Transform the data
transformed_data = pca.predict(features_train)

# Add the response column back to the transformed data
transformed_data[y] = features_train[y]

# Update the predictors list
x = transformed_data.col_names
x.remove(y)

# Initialize AutoML
aml = H2OAutoML(max_runtime_secs_per_model=3600, max_models=50, nfolds=5 ,balance_classes= True ,sort_metric='AUCPR', seed=1)

# Train model
aml.train(x=x, y=y, training_frame=transformed_data)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)


pca Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
pca prediction progress: |

c:\Users\byan\eeg_project\.venv\Lib\site-packages\h2o\job.py:81: UserWarning: _train: Dataset used may contain fewer number of rows due to removal of rows with NA/missing values.  If this is not desirable, set impute_missing argument in pca call to TRUE/True/true/... depending on the client language.
  warnings.warn(w)


███████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
17:25:33.168: AutoML: XGBoost is not available; skipping it.

██████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_bca0b0e9344fd353956fe5f7149e470c> was cancelled by the user.

In [5]:
best_model = aml.get_best_model(criterion='mean_per_class_error')

In [6]:
best_performance = best_model.model_performance(features_test)

In [8]:
best_performance

ModelMetricsBinomial: stackedensemble
** Reported on test data. **

MSE: 0.12484460590008649
RMSE: 0.3533335618082246
LogLoss: 0.41756200422251055
Mean Per-Class Error: 0.5
AUC: 0.5
AUCPR: 0.1443167305236271
Gini: 0.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.18112969135926255
         normal    seizure    Error    Rate
-------  --------  ---------  -------  ---------------
normal   0         1340       1        (1340.0/1340.0)
seizure  0         226        0        (0.0/226.0)
Total    0         1566       0.8557   (1340.0/1566.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.18113      0.252232  0
max f2                       0.18113      0.45749   0
max f0point5                 0.18113      0.174114  0
max accuracy                 0.18113      0.144317  0
max precision                0.18113      0.144317  0
max recall                   0.18113      1         0
max specificity              0.18113      0         0
max absolute_mcc             0.18113      0         0
max min_per_class_accuracy   0.18113      0         0
max mean_per_class_accuracy  0.18113      0.5       0
max tns                      0.18113      0         0
max fns                      0.18113      0         0
max fps                      0.18113      1340      0
max tps                      0.18113      226       0
max tnr                      0.18113      0         0
max fnr                      0.18113      0         0
max fpr                      0.18113      1         0
max tpr                      0.18113      1         0

Gains/Lift Table: Avg response rate: 14.43 %, avg score: 18.11 %
group    cumulative_data_fraction    lower_threshold    lift    cumulative_lift    response_rate    score    cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain    cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ------  -----------------  ---------------  -------  --------------------------  ------------------  --------------  -------------------------  ------  -----------------  --------------------
1        1                           0.18113            1       1                  0.144317         0.18113  0.144317                    0.18113             1               1                          0       0                  0